In [ ]:
%load_ext autoreload
%autoreload 2

import uproot
import matplotlib

import matplotlib.pyplot as plt
import matplotlib.colors as colors
import matplotlib.cbook as cbook
import numpy as np
import pandas as pd
from decimal import Decimal
from scipy.stats import norm
from scipy.optimize import curve_fit
from scipy import stats, interpolate
import scipy
import warnings
import datetime as dt
import landau
from multiprocessing import Pool, cpu_count

from matplotlib.patches import Rectangle

import importlib
import os
from tqdm.auto import tqdm

# local imports
from lib.constants import *

from scipy.integrate import quad, tplquad
from scipy.special import erf
from scipy.interpolate import CubicSpline

In [ ]:
dosave = True
plotqual="yznorm_runA_ind1_"
tqual = "Run A Middle Induction Plane "
savedir = "plots_10_30_23/%s" % plotqual
plt.rcParams.update({'font.size': 14})
datadir = "/icarus/data/users/gputnam/calib-data/outputs/"
# datadir = "/icarus/data/users/gputnam/MCP2022G/mc/cosmic3/"

isMC = False
dataf = datadir + "P1_scaleYZ_RunA.txt"

overlay_faults = False
drawfid = False

PLANE = 1

In [ ]:
yz_ybin = np.linspace(-180, 130, 32)
yz_ylos = yz_ybin[:-1]
yz_yhis = yz_ybin[1:]
yz_ys = (yz_ylos + yz_yhis) / 2.

yz_zbin = np.linspace(-900, 900, 181)
yz_zlos = yz_zbin[:-1]
yz_zhis = yz_zbin[1:]
yz_zs = (yz_zlos + yz_zhis) / 2.

In [ ]:
if isMC:
    yz_ybin = np.linspace(-180, 120, 11)
    yz_ylos = yz_ybin[:-1]
    yz_yhis = yz_ybin[1:]
    yz_ys = (yz_ylos + yz_yhis) / 2.

    yz_zbin = np.linspace(-900, 900, 61)
    yz_zlos = yz_zbin[:-1]
    yz_zhis = yz_zbin[1:]
    yz_zs = (yz_zlos + yz_zhis) / 2.

In [ ]:
tpcs = ["EE", "EW", "WE", "WW"]

In [ ]:
zy_mpvs = {}
zy_mpverrs = {}
zy_Ns = {}

for tpc in tpcs:
    zy_mpvs[tpc] = []
    zy_mpverrs[tpc] = []
    zy_Ns[tpc] = []
    for _ in yz_ys:
        zy_mpvs[tpc].append([])
        zy_mpverrs[tpc].append([])
        zy_Ns[tpc].append([])
        
        for _ in yz_zs:
            zy_mpvs[tpc][-1].append(-1)
            zy_mpverrs[tpc][-1].append(-1)
            zy_Ns[tpc][-1].append(-1)

In [ ]:
with open(dataf) as f:
    next(f)
    for line in f:
        dat = line.rstrip("\n").split("\t")
        tpc = dat[0]
        iy = int(dat[1])
        iz = int(dat[2])
        mpv = float(dat[4])
        merr = float(dat[5])
        N = int(float(dat[6]))
        
        zy_mpvs[tpc][iy][iz] = mpv
        zy_mpverrs[tpc][iy][iz] = merr
        zy_Ns[tpc][iy][iz] = N
        

In [ ]:
central_mpv = np.mean([np.nanmean(m) for m in zy_mpvs.values()])

In [ ]:
for ifig, tpc in enumerate(tpcs):
    plt.figure(ifig, figsize=(20, 4))
    
    mpvs = np.array(zy_mpvs[tpc]).flatten()
    err_ratio = np.array(zy_mpverrs[tpc]).flatten() / mpvs

    if PLANE == 0:
        vmin = 0.85
        vmax = 1.15
    elif PLANE == 1:
        vmin = 0.8
        vmax = 1.25
    elif PLANE == 2:
        vmin = 0.85
        vmax = 1.15
        
    plt.imshow(zy_mpvs[tpc]/central_mpv, extent=[-900, 900, -180, 130], origin="lower", vmin=vmin, vmax=vmax)
    cbar = plt.colorbar(pad=0.01)
    plt.ylabel("Detector Y [cm]")
    plt.xlabel("Detector Z [cm]")
    plt.title("%sTPC %s" % (tqual, tpc), loc="right")
#     cbar.set_label('10-Wire MPV dQ/dx [ADC/cm]', rotation=270, labelpad=15)
    cbar.set_label('Relative Charge Scale', rotation=270, labelpad=15)
    
    flangecolor = "orange"
    flangeline =":"
    flangelwidth = 2
    
    def flange_plot(z0, angle, col):
        y0 = 134.95
        dw = 1e3
        plt.plot([z0, z0 + dw*np.sin(angle)], [y0, y0 - dw*np.cos(angle)], 
                 color="black" if col else "deeppink", linestyle=":", linewidth=2)

    def flange_plot(y0, z0, y1, z1, col):
        plt.plot([z0, z1], [y0, y1], 
                 color="black" if col else "deeppink", linestyle=":", linewidth=2)

    s = -1
    flange_plot(134.950000, -894.951000, -181.850000, -712.046000, tpc[1] == "W")
    flange_plot(134.950000, -795.185000, -181.850000, -612.280000, tpc[1] == "W")
    flange_plot(134.950000, -695.418000, -181.850000, -512.514000, tpc[1] == "W")
    flange_plot(134.950000, -595.652000, -181.850000, -412.748000, tpc[1] == "W")
    flange_plot(134.950000, -495.886000, -181.850000, -312.982000, tpc[1] == "W")
    flange_plot(134.950000, -396.120000, -181.850000, -213.215000, tpc[1] == "W")
    flange_plot(134.950000, -296.354000, -181.850000, -113.449000, tpc[1] == "W")
    flange_plot(134.950000, -196.588000, -181.850000, -13.683200, tpc[1] == "W")
    flange_plot(134.950000, -96.821600, -181.850000, 86.082900, tpc[1] == "W")
    flange_plot(134.950000, 2.944490, -181.850000, 185.849000, tpc[1] == "W")
    flange_plot(134.950000, 102.711000, -181.850000, 285.615000, tpc[1] == "W")
    flange_plot(134.950000, 202.477000, -181.850000, 385.381000, tpc[1] == "W")
    flange_plot(134.950000, 302.243000, -181.850000, 485.147000, tpc[1] == "W")
    flange_plot(134.950000, 402.009000, -181.850000, 584.914000, tpc[1] == "W")
    flange_plot(134.950000, 501.775000, -181.850000, 684.680000, tpc[1] == "W")
    flange_plot(134.950000, 601.541000, -181.850000, 784.446000, tpc[1] == "W")
    flange_plot(134.950000, 701.307000, -181.850000, 884.212000, tpc[1] == "W")
    flange_plot(134.950000, 801.073000, -27.650000, 894.951000, tpc[1] == "W")
    
    s = 1
    flange_plot(134.950000, -800.727000, -28.250000, -894.951000, tpc[1] == "E")
    flange_plot(134.950000, -700.961000, -181.850000, -883.866000, tpc[1] == "E")
    flange_plot(134.950000, -601.195000, -181.850000, -784.099000, tpc[1] == "E")
    flange_plot(134.950000, -501.429000, -181.850000, -684.333000, tpc[1] == "E")
    flange_plot(134.950000, -401.663000, -181.850000, -584.567000, tpc[1] == "E")
    flange_plot(134.950000, -301.896000, -181.850000, -484.801000, tpc[1] == "E")
    flange_plot(134.950000, -202.130000, -181.850000, -385.035000, tpc[1] == "E")
    flange_plot(134.950000, -102.364000, -181.850000, -285.269000, tpc[1] == "E")
    flange_plot(134.950000, -2.598080, -181.850000, -185.503000, tpc[1] == "E")
    flange_plot(134.950000, 97.168100, -181.850000, -85.736500, tpc[1] == "E")
    flange_plot(134.950000, 196.934000, -181.850000, 14.029600, tpc[1] == "E")
    flange_plot(134.950000, 296.700000, -181.850000, 113.796000, tpc[1] == "E")
    flange_plot(134.950000, 396.466000, -181.850000, 213.562000, tpc[1] == "E")
    flange_plot(134.950000, 496.233000, -181.850000, 313.328000, tpc[1] == "E")
    flange_plot(134.950000, 595.999000, -181.850000, 413.094000, tpc[1] == "E")
    flange_plot(134.950000, 695.765000, -181.850000, 512.860000, tpc[1] == "E")
    flange_plot(134.950000, 795.531000, -181.850000, 612.626000, tpc[1] == "E")
    flange_plot(134.950000, 895.531000, -181.850000, 712.626000, tpc[1] == "E")

    if drawfid:
        plt.gca().add_patch(Rectangle((-1000, -190), 200, 350, facecolor=colors.to_rgba('red', 0.4)))
        plt.gca().add_patch(Rectangle((800, -190), 200, 350, facecolor=colors.to_rgba('red', 0.4)))
        
        plt.gca().add_patch(Rectangle((-800, 114), 1600, 20, facecolor=colors.to_rgba('red', 0.4)))
        plt.gca().add_patch(Rectangle((-800, -180), 1600, 20, facecolor=colors.to_rgba('red', 0.4)))
        
        #plt.gca().add_patch(Rectangle((-10, -160), 20, 274, facecolor=colors.to_rgba('red', 0.4)))
        
        if tpc == "EW":
            plt.gca().add_patch(Rectangle((700, -160), 100, 160, facecolor=colors.to_rgba('red', 0.4)))
        if tpc == "WW":
            plt.gca().add_patch(Rectangle((10, 80), 790, 34, facecolor=colors.to_rgba('red', 0.4)))
            
    
    plt.xlim([-900, 900])
    plt.ylim([-180, 130])
    
    plt.axvline(0, color="red", linestyle="--")
    plt.axvline(299.3, color="red", linestyle="--")
    plt.axvline(-299.3, color="red", linestyle="--")
    plt.axvline(598.5, color="red", linestyle="--")
    plt.axvline(-598.5, color="red", linestyle="--")
    
    plt.text(0.01, 1.05, "Collection Plane Flanges / ", transform=plt.gca().transAxes)
    plt.text(0.205, 1.05, "Induction 2 Plane Flanges", transform=plt.gca().transAxes, color="deeppink")
    plt.text(0.40, 1.05, " / Wire Supports", transform=plt.gca().transAxes, color="red")
    
    if overlay_faults and tpc == "WE":
        plt.text(0.025, 0.15, "Induction 2\nWire Bias Grounded", 
                 color="white", transform=plt.gca().transAxes,
                rotation=-60)
    if overlay_faults and tpc == "WW":
        plt.text(0.02, 0.65, "Ind. 2\nGrounded", 
                 color="white", transform=plt.gca().transAxes, horizontalalignment="center",
                rotation=60, fontsize=11)
    if overlay_faults and tpc == "WW":
        plt.text(0.7125, 0.8125, "Dangling\nCable", color="white", transform=plt.gca().transAxes)
        
    if dosave: 
        plt.savefig(savedir + "mpv_yz%s.pdf" % (tpc), bbox_inches="tight")
        plt.savefig(savedir + "mpv_yz%s.png"% (tpc), bbox_inches="tight")

In [ ]:
#     if PLANE == 2:
#         if tpc == "EE":
#             vmin = 575
#             vmax = 800
#         elif tpc == "EW":
#             vmin = 645
#             vmax = 790
#         elif tpc == "WE":
#             vmin = 590
#             vmax = 765
#         elif tpc == "WW":
#             vmin = 610
#             vmax = 790
#     elif PLANE == 1:
#         if tpc == "EE":
#             vmin = 540
#             vmax = 865
#         elif tpc == "EW":
#             vmin = 570
#             vmax = 820
#         elif tpc =="WE":
#             vmin = 545
#             vmax = 735
#         elif tpc =="WW":
#             vmin = 550
#             vmax = 760
#     elif PLANE == 0:
#         if tpc == "EE":
#             vmin = 630
#             vmax = 795
#         elif tpc == "EW":
#             vmin = 670
#             vmax = 875
#         elif tpc =="WE":
#             vmin = 645
#             vmax = 795
#         elif tpc =="WW":
#             vmin = 555
#             vmax = 840
#     else:
#         vmin = mpvs[err_ratio < 0.005].min()
#         vmax = mpvs[err_ratio < 0.005].max()
#         print(vmin, vmax)

In [ ]:
for ifig, tpc in enumerate(tpcs):
    plt.figure(ifig, figsize=(20, 4))
    
    #vmin = mpvs[err_ratio < 0.01].min()
    #vmax = mpvs[err_ratio < 0.01].max()
    
    plt.imshow(zy_Ns[tpc], extent=[-900, 900, -180, 130], origin="lower") #, vmin=vmin, vmax=vmax)
    plt.colorbar(pad=0.01)
    #plt.gca().set_aspect(1./6)
    plt.ylabel("Detector Y [cm]")
    plt.xlabel("Detector Z [cm]")
    plt.title("%sNumber of Hits, TPC %s" % (tqual, tpc))
    if dosave: 
        plt.savefig(savedir + "N_yz%s.pdf"% tpc)
        plt.savefig(savedir + "N_yz%s.png" % tpc)

In [ ]:
for ifig, tpc in enumerate(tpcs):
    plt.figure(ifig, figsize=(20, 4))
    
    #vmin = mpvs[err_ratio < 0.01].min()
    #vmax = mpvs[err_ratio < 0.01].max()
    
    plt.imshow(100*np.array(zy_mpverrs[tpc]) / np.array(zy_mpvs[tpc]) , extent=[-900, 900, -180, 130], origin="lower",
              vmin=0, vmax=3)
    plt.colorbar(pad=0.01)
    #plt.gca().set_aspect(1./6)
    plt.ylabel("Detector Y [cm]")
    plt.xlabel("Detector Z [cm]")
    plt.title("%sPercent Error on MPV, TPC %s" % (tqual, tpc))
    if dosave: 
        plt.savefig(savedir + "err_yz%s.pdf" % tpc)
        plt.savefig(savedir + "err_yz%s.png" % tpc)
        

In [ ]:
if isMC:
    bins = np.linspace(670, 700, 16)
else:
    bins = np.linspace(600, 850, 26)

for ifig, tpc in enumerate(tpcs):
    plt.figure(ifig)
    dat = np.array(zy_mpvs[tpc]).flatten()
    _ = plt.hist(dat, bins=bins)
    plt.title("%sTPC %s" %(tqual, tpc))
    plt.ylabel("# of 10cm Z-Y bins")
    plt.xlabel("MPV dQ/dx [ADDC/cm]")
    
    if dosave: plt.savefig(savedir + "scale_dist_tpc%s.pdf" % (tpc))

In [ ]:
bin_is_fid = np.outer((yz_ys < 114) & (yz_ys > -160), np.abs(yz_zs) < 800)

In [ ]:
for ifig, tpc in enumerate(tpcs):
    plt.figure(ifig)
    dat = np.array(zy_mpvs[tpc])[bin_is_fid].flatten()
    _ = plt.hist(dat, bins=bins)
    plt.title("%sTPC %s" %(tqual, tpc))
    plt.ylabel("# of 10cm Z-Y bins")
    plt.xlabel("MPV dQ/dx [ADDC/cm]")
    
    if dosave: plt.savefig(savedir + "scale_dist_fid_tpc%s.pdf" % (tpc))

In [ ]:
for i, tpc in enumerate(tpcs):
    plt.figure(i)
    
    mpvs = np.mean(np.array(zy_mpvs[tpc]), axis=-1)
    plt.plot(yz_ys, mpvs)